## Heuristic learning

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [35]:
dataset = np.load("star_battle_dataset.npz", allow_pickle=True)
puzzles = dataset["puzzles"]
solutions = dataset["solutions"]
# Check the shape of the dataset
print("Puzzles shape:", puzzles.shape)
print("Solutions shape:", solutions.shape)

Puzzles shape: (1200, 10, 10)
Solutions shape: (1200, 10, 10)


## data pre processing

In [40]:
# augment data for larger dataset
def augment_data(puzzles, solutions):
    augmented_puzzles = []
    augmented_solutions = []
    
    for i in range(len(puzzles)):
        puzzle = puzzles[i]
        solution = solutions[i]
        
        # original puzzle
        augmented_puzzles.append(puzzle)
        augmented_solutions.append(solution)
        
        # horizontal Flip
        augmented_puzzles.append(np.fliplr(puzzle))
        augmented_solutions.append(np.fliplr(solution))
        
        # vertical Flip
        augmented_puzzles.append(np.flipud(puzzle))
        augmented_solutions.append(np.flipud(solution))
        
        # rotate 90 degrees
        augmented_puzzles.append(np.rot90(puzzle, k=1))
        augmented_solutions.append(np.rot90(solution, k=1))
        
        # rotate 180 degrees
        augmented_puzzles.append(np.rot90(puzzle, k=2))
        augmented_solutions.append(np.rot90(solution, k=2))
        
        # rotate 270 degrees
        augmented_puzzles.append(np.rot90(puzzle, k=3))
        augmented_solutions.append(np.rot90(solution, k=3))
    
    return np.array(augmented_puzzles), np.array(augmented_solutions)

# apply augmentation to your dataset
augmented_puzzles, augmented_solutions = augment_data(puzzles, solutions)

print("Original puzzle count:", len(puzzles))
print("Augmented puzzle count:", len(augmented_puzzles))


Original puzzle count: 1200
Augmented puzzle count: 7200


In [42]:
# Reshape puzzles and solutions if necessary (ensure each puzzle is a 2D grid)
augmented_puzzles = puzzles[..., np.newaxis] # Assuming 10x10 grids, change if needed
augmented_solutions = solutions[..., np.newaxis]  # Same for solutions

# Normalize puzzle values between 0 and 1 (if necessary)
augmented_puzzles = puzzles.astype(np.float32) / 9.0

augmented_solutions = solutions.astype(np.float32) 

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(augmented_puzzles, augmented_solutions, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}, {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, {y_test.shape}")


Training data shape: (960, 10, 10, 1), (960, 10, 10, 1)
Testing data shape: (240, 10, 10, 1), (240, 10, 10, 1)


## build model

In [48]:
# Build the CNN model
model = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(10, 10, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (1, 1), activation='relu', padding='same'),
    layers.BatchNormalization(),

    layers.Flatten(),
    layers.Dense(100, activation='sigmoid'),  # Predicts for all 100 cells
    layers.Reshape((10, 10, 1))  # reshape back to 10x10 grid
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()



Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 10, 10, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 10, 10, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 5, 5, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 5, 5, 64)          36928     
                                                                 
 batch_normalization_1 (Batc  (None, 5, 5, 64)         256       
 hNormalization)                                                 
                                                      

## train model

In [49]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
30/30 [==============================] - 3s 41ms/step - loss: 0.6321 - accuracy: 0.7105 - val_loss: 0.6663 - val_accuracy: 0.6349
Epoch 2/20
30/30 [==============================] - 0s 11ms/step - loss: 0.5165 - accuracy: 0.7885 - val_loss: 0.6531 - val_accuracy: 0.6305
Epoch 3/20
30/30 [==============================] - 0s 11ms/step - loss: 0.4893 - accuracy: 0.7966 - val_loss: 0.6357 - val_accuracy: 0.6641
Epoch 4/20
30/30 [==============================] - 0s 12ms/step - loss: 0.4729 - accuracy: 0.8001 - val_loss: 0.6262 - val_accuracy: 0.6658
Epoch 5/20
30/30 [==============================] - 0s 12ms/step - loss: 0.4569 - accuracy: 0.8030 - val_loss: 0.6222 - val_accuracy: 0.6737
Epoch 6/20
30/30 [==============================] - 0s 12ms/step - loss: 0.4422 - accuracy: 0.8077 - val_loss: 0.6201 - val_accuracy: 0.6817
Epoch 7/20
30/30 [==============================] - 0s 11ms/step - loss: 0.4264 - accuracy: 0.8136 - val_loss: 0.6219 - val_accuracy: 0.6806
Epoch 8/20
30

## evaluate model

In [50]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")


8/8 [==============================] - 0s 4ms/step - loss: 0.6561 - accuracy: 0.7250
Test Accuracy: 0.7250


In [2]:
dataset = np.load("star_battle_dataset.npz", allow_pickle=True)
puzzles = dataset["puzzles"]
solutions = dataset["solutions"]
# Check the shape of the dataset
print("Puzzles shape:", puzzles.shape)
print("Solutions shape:", solutions.shape)

ModuleNotFoundError: No module named 'numpy._core'